# Scraping posts data from Facebook Page for analysis

We use Facebook graph API v2.11 in this program. 

In [ ]:
#Import Python libraries
import requests
import json
import datetime
import csv
import time

Accessing or Pulling the data from Facebook using Graph API requires an access token, and instructions for creation of access token can be found [here](https://developers.facebook.com/docs/facebook-login/access-tokens#apptokens).

In [ ]:
#access_token 
app_id = 'your_ap_id_here'
app_secret = 'your_secret_here'
access_token = app_id + '|' + app_secret

Let us do our analysis with Rolls Royce page. 

In [ ]:
page_name = 'rollsroycemotorcars'

We need to obtain the Facebook Page ID for pulling in the data using FB Graph API (v 2.11)
Let us define a function to obtain the fb page id, using page name. 

The following function is used to gather the facebook page id. 

In [ ]:
def getpageid (page_name):
    base = "https://graph.facebook.com/v2.11/"
    queryurl = base + page_name + '/?access_token=' + access_token    
    page_info = requests.get(queryurl).json()
    return page_info['id']

In [ ]:
page_id = getpageid(page_name)
#print(page_id)

A function to obtain the posts of a facebook page. 

In [ ]:
def fbposts(fburl):
    fb_page_posts = req_until_success(fburl).json()
    page_posts = fb_page_posts['data']
    
    end_file = fb_page_posts['paging']
    
    if 'paging' in fb_page_posts.keys():
        end_file = fb_page_posts['paging']
        next_url = str(end_file['next']).replace('%2C', ',').replace('%28', '(').replace('%29', ')')
    else:
        next_url = 'End'
    return page_posts, next_url 

In [ ]:
#Function: Extraction of Required Data
def postextract(posts_data):
    a = {}
    for i in range (len(posts_data)):
        post_id = '' if 'id' not in posts_data[i].keys() else posts_data[i]['id']
        link = '' if 'link' not in posts_data[i].keys() else posts_data[i]['link']
        message  = 'NA' if 'message' not in posts_data[i].keys() else posts_data[i]['message']
        perma_link = '' if 'permalink_url' not in posts_data[i].keys() else posts_data[i]['permalink_url']
        post_type = '' if 'type' not in posts_data[i].keys() else posts_data[i]['type']
        if 'created_time' in posts_data[i].keys():
            time_info = posts_data[i]['created_time']
            time_stamp = time_info.split('T')[1].split('+0000')[0]
            date_stamp = time_info.split('T')[0]
        else:
            time_stamp, date_stamp = "NA", "NA"
        comments = '' if 'comments' not in posts_data[i].keys() else posts_data[i]['comments']['summary']['total_count']
        shares = '' if 'shares' not in posts_data[i].keys() else posts_data[i]['shares']['count']
        likes = '' if 'likes' not in posts_data[i].keys() else posts_data[i]['likes']['summary']['total_count']
        reactions_none = '' if 'reactions_none' not in posts_data[i].keys() else posts_data[i]['reactions_none']['summary']['total_count']
        reactions_love = '' if 'reactions_love' not in posts_data[i].keys() else posts_data[i]['reactions_love']['summary']['total_count']
        reactions_wow = '' if 'reactions_wow' not in posts_data[i].keys() else posts_data[i]['reactions_wow']['summary']['total_count']
        reactions_haha = '' if 'reactions_haha' not in posts_data[i].keys() else posts_data[i]['reactions_haha']['summary']['total_count']        
        reactions_sad = '' if 'reactions_sad' not in posts_data[i].keys() else posts_data[i]['reactions_sad']['summary']['total_count']
        reactions_angry = '' if 'reactions_angry' not in posts_data[i].keys() else posts_data[i]['reactions_angry']['summary']['total_count']
        a[i] = [date_stamp, time_stamp, post_id, perma_link, link, message, post_type,   
                 comments, shares, reactions_none, likes, reactions_love, 
                 reactions_wow, reactions_haha, reactions_sad, reactions_angry]
    return a 


In [ ]:
#CSV Writer
#This function writes all the extracted posts into CSV file.  
def tocsv(name,posts_list):
    outputfile = dirname + "/" + page_name + '_' + 'posts' + name + '.csv'
    print('working on {}'.format(name))
    with open(outputfile, 'w', encoding = 'utf-8-sig', newline='') as outfile:
        writer = csv.writer(outfile)
        Header = ("Date", "Time", "ID" , "PermaLink" , "Link", "Message", "Post Type", 
                        "Comments", "Shares", "Total Reactions", "Likes",
                        "Love","Wow","Haha", "Sad","Angry")
        writer.writerow(Header)
        for k in posts_list.keys():
            writer.writerow(posts_list[k])

In [ ]:
#Main Function
param = '/posts/?fields=message,link,permalink_url,created_time,type,name,id,comments.limit(0).summary(true),shares,likes.limit(0).summary(true),reactions.type(NONE).limit(0).summary(total_count).as(reactions_none),reactions.type(LIKE).limit(0).summary(total_count).as(reactions_like),reactions.type(LOVE).limit(0).summary(total_count).as(reactions_love),reactions.type(WOW).limit(0).summary(total_count).as(reactions_wow),reactions.type(HAHA).limit(0).summary(total_count).as(reactions_haha),reactions.type(SAD).limit(0).summary(total_count).as(reactions_sad),reactions.type(ANGRY).limit(0).summary(total_count).as(reactions_angry)&limit=100&access_token='
posts_queryurl = base + page_id + param + access_token
posts, next_url = fbposts(posts_queryurl)
a_posts = postextract(posts)
tocsv('Part1', a_posts)
z = 1
endofposts = False 

#Iterate and pullin all the posts within the date range
if next_url != 'End':
    next_url_part_c = next_url
    while next_url_part_c != 'End':
        z += 1
        kl  = "Part" + str(z) 
        next_posts_part,next_url_part = fbposts(next_url_part_c)
        b_posts = postextract(next_posts_part)
        tocsv(str(kl), b_posts)
        next_url_part_c = next_url_part

In [ ]:
#Merge CSV files
to_merge = glob.glob(dirname+"/*.csv")
to_merge.sort()
header_saved = False
outputfile_m = dirname + "/" + page_name + '_' + 'posts'+ '_' +'merged.csv'
with open(outputfile_m,'w', encoding = 'utf-8-sig') as mergedfile:
    for filename in to_merge:
        with open(filename, encoding = 'utf-8-sig') as infiles:
            header = next(infiles)
            if not header_saved:
                mergedfile.write(header)
                header_saved = True
            for line in infiles:
                mergedfile.write(line)

In [ ]:
print('Posts of page {} from {} to {} are obtained'.format(page_name, start_date,end_date))